# 射出成型機
処理の流れ：npyをloadしてショット毎のcsvに分割⇒csvを加工しつつpklに変換⇒pklをloadしてElasticsearchに格納

  Copyright(c) 2021 UNIADEX, Ltd. All Rights Reserved.  
  CONFIDENTIAL  
  Author: UNIADEX, Ltd.

In [1]:
# ライブラリインポート
import os, sys
sys.path.append(os.path.join(os.getcwd(), '../src/'))
from cut_out_shot.cut_out_shot import CutOutShot
from utils import common
import logging
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from data_importer.data_importer import DataImporter

## npyをloadしてショット毎のcsvファイルにする
既にcsvがある場合は実行不要

In [ ]:
di = DataImporter()
target_file_path = "/home/ymiyamoto5/h-one-experimental-system/shared/data/all_sensors_201905standardized.npy"
target_date = "20210701180000"
di.split_np_array_by_shot(target_file_path, target_date)

In [ ]:
# データディレクトリの一覧
!ls /home/ymiyamoto5/h-one-experimental-system/shared/data/20210701180000

## csvファイルを加工しつつpklファイルにする
既にpklファイルがある場合は実行不要

In [ ]:
# read from csv in dir
target_date = "20210701180000"
start_time = datetime(2021, 7, 1, 18, 0, 0)
# 何列目を使うか。
use_cols = [2, 6, 7, 11, 57]
cols_name = ["load01", "load02", "load03", "load04", "displacement"]
sampling_interval = 1000 # 1k
di = DataImporter()
# csvを加工してpklに出力
di.process_csv_files(target_date=target_date,
           exists_timestamp=False,
           start_time=start_time,
           use_cols=use_cols,
           cols_name=cols_name,
           sampling_interval=sampling_interval
          )

## (オプション) pklファイルの確認

In [ ]:
file_path = "/home/ymiyamoto5/h-one-experimental-system/shared/data/20210701180000/0000.pkl"
df = pd.read_pickle(file_path)
df.head()

In [ ]:
len(df)

## pklファイルをloadしてElasticsearchに出力

In [2]:
# pklファイルからElasticsearchにインポート（切り出しなし）
target_date = "20210701180000"
di = DataImporter()
di.import_by_shot_pkl(target_date)

shots_index bulk insert starting...
shots_index bulk insert finished.
shots_meta_index bulk insert finished.


## (オプション）events_indexの作成
既に存在する場合は実行不要

In [ ]:
# （なければ）events_index作成
from utils.create_events_index import create_events_index
prefix = "events-"
start_datetime_jst = datetime(2021, 7, 1, 18, 0, 0, 0)
create_events_index(start_datetime_jst)